### Illumination Correction 

In [ ]:
pip install basicpy

In [ ]:
# pip install tifffile

In [1]:
import h5py
from PIL import Image
import io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path
import numpy as np
import os
import glob
import pandas as pd
import basicpy
import tifffile as tiff
from basicpy import BaSiC
from skimage import io

/home/jz3553_columbia_edu/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [9]:
import concurrent.futures

#### Copy all CF and WF tiles for all cycles 

In [2]:
fguide = pd.read_csv("fileguide_S17.csv")
fguide.dropna(inplace=True)
fguide.reset_index(drop=True, inplace=True)
fguide.rename(columns={"Unnamed: 0": "cycle"}, inplace=True)
fguide.rename(columns={"WF (750)": "WF"}, inplace=True)
print(fguide)

       cycle                   CF                   WF
0    Cycle 1     Cycle_2023-02-27   Cycle_2023-02-27_1
1    Cylce 2   Cycle_2023-02-27_2   Cycle_2023-02-27_4
2    Cycle 3   Cycle_2023-02-27_5   Cycle_2023-02-27_6
3    Cycle 4   Cycle_2023-02-27_7   Cycle_2023-02-27_8
4    Cycle 5   Cycle_2023-02-27_9  Cycle_2023-02-27_10
5    Cycle 6  Cycle_2023-02-27_11  Cycle_2023-02-27_12
6   Cycle 7   Cycle_2023-02-27_13  Cycle_2023-02-27_14
7    Cycle 8  Cycle_2023-02-27_15     Cycle_2023-02-28
8    Cycle 9  Cycle_2023-02-28_21  Cycle_2023-02-28_22
9   Cycle 10  Cycle_2023-02-28_29  Cycle_2023-02-28_30
10  Cycle 11  Cycle_2023-02-28_25  Cycle_2023-02-28_26
11  Cycle 12  Cycle_2023-02-28_27  Cycle_2023-02-28_28
12  Cycle 13   Cycle_2023-02-28_9  Cycle_2023-02-28_10
13  Cycle 14  Cycle_2023-02-28_11  Cycle_2023-02-28_12
14  Cycle 15  Cycle_2023-02-28_13  Cycle_2023-02-28_14
15  Cycle 16  Cycle_2023-02-28_15  Cycle_2023-02-28_16
16  Cycle 17  Cycle_2023-02-28_17  Cycle_2023-02-28_18
17  Cycle 

In [ ]:
import subprocess

In [ ]:
# Loop through each cycle
for index, row in fguide.iterrows():
    cycle = row['cycle']
    cf_guide = row['CF']
    wf_guide = row['WF']
    
    # Build the search strings
    cf_search_string = f'gsutil ls gs://dli-codex-firstround/**{cf_guide}_F**'
    wf_search_string = f'gsutil ls gs://dli-codex-firstround/**{wf_guide}_F**'
    
    # copy files 
    cf_files = subprocess.getoutput(f'gsutil -m cp {cf_search_string} all_cycles')
    wf_files = subprocess.getoutput(f'gsutil -m cp {wf_search_string} all_cycles')
    
#     # Execute the gsutil command and collect the results
#     cf_files = subprocess.getoutput(cf_search_string)
#     wf_files = subprocess.getoutput(wf_search_string)
    
#     # Print out or process the files found
#     print(f'Files for {cycle} - CF Guide:')
#     print(cf_files)
#     print(f'Files for {cycle} - WF Guide:')
#     print(wf_files)

    print(f'Files copied and processed for cycle {cycle}')

#### Apply illumination correction through Basicpy

In [3]:
# edited both functions to handle different input sizes (could be 2 or 3 channels depending on WF or CF)
def to_numpy(filename):
    ims = h5py.File(filename, 'r')
    data = ims['DataSet']['ResolutionLevel 0']['TimePoint 0']
    
    channels = []
    for key in data.keys():
        if 'Channel' in key:
            channel_data = data[key]['Data'][()]
            channels.append(channel_data[0])
    
    ims.close()
    return channels

def normalize(image):
    MAX = np.quantile(image, 0.99)
    MIN = np.quantile(image, 0.05)
    # Clip the values
    image_clipped = np.clip(image, MIN, MAX)
    # Normalize the clipped image
    normalized_image = (image_clipped - MIN) / (MAX - MIN)
    return normalized_image

In [12]:
# Function to process each file
def process_file(filename):
    file_path = os.path.join(directory, filename)
    img = to_numpy(file_path)
    img = np.array(img)
    
    corrector = basicpy.BaSiC()
    corrected_image = corrector.fit_transform(img)
    corrected_image = normalize(corrected_image)

    output_filename = os.path.splitext(filename)[0] + ".tiff"
    corrected_image_path = os.path.join(output_dir, output_filename)

    tiff.imwrite(corrected_image_path, corrected_image)
    return f"Illumination correction applied and saved to {corrected_image_path}"

In [13]:
# Main parallel processing block
if __name__ == "__main__":
    directory = 'all_cycles'
    output_dir = "corr_all"

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # List of all .ims files
        files = [f for f in os.listdir(directory) if f.lower().endswith('.ims')]
        results = executor.map(process_file, files)

        for result in results:
            print(result)

Illumination correction applied and saved to corr_all/Cycle_2023-02-28_17_F009.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_F037.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_30_F057.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_15_F044.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_11_F098.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_11_F092.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_8_F086.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_11_F055.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_21_F083.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_12_F007.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_1_F116.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_12_F062.ti

Reweighting did not converge.


Illumination correction applied and saved to corr_all/Cycle_2023-02-27_2_F027.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_27_F015.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_6_F092.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_6_F093.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_9_F044.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_2_F006.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_12_F117.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_9_F035.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_6_F049.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_26_F098.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_27_F084.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_11_F093.tiff

Reweighting did not converge.


Illumination correction applied and saved to corr_all/Cycle_2023-02-28_22_F040.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_27_F116.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_F052.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_18_F117.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_25_F051.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_12_F067.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_13_F018.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_14_F100.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_F021.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_6_F010.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_14_F053.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_6_F018.tiff


...

Illumination correction applied and saved to corr_all/Cycle_2023-02-28_27_F086.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_12_F026.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_18_F060.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_15_F074.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_26_F119.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_17_F104.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_14_F016.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_12_F005.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_25_F103.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_21_F121.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_14_F122.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_17_F0

Reweighting did not converge.


Illumination correction applied and saved to corr_all/Cycle_2023-02-27_2_F052.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_9_F032.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_10_F005.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_1_F076.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_30_F023.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_5_F091.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_9_F037.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_30_F064.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_20_F095.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_21_F059.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_28_F006.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_7_F051.tif

Reweighting did not converge.


Illumination correction applied and saved to corr_all/Cycle_2023-02-28_12_F006.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_11_F038.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_13_F028.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_29_F058.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_10_F062.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_27_F100.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_27_F054.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_10_F120.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_30_F021.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_7_F066.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-28_9_F002.tiff
Illumination correction applied and saved to corr_all/Cycle_2023-02-27_1_F062.

In [ ]:
# Illumination correction applied and saved to all_cycles_output/Cycle_2023-02-28_27_F015.ims - Reweighting did not converge.

# Illumination correction applied and saved to all_cycles_output/Cycle_2023-02-28_27_F116.ims - Reweighting did not converge.

# Illumination correction applied and saved to all_cycles_output/Cycle_2023-02-28_27_F086.ims - Reweighting did not converge.

# Illumination correction applied and saved to all_cycles_output/Cycle_2023-02-28_17_F017.ims - Reweighting did not converge.

# Illumination correction applied and saved to all_cycles_output/Cycle_2023-02-27_15_F112.ims - Reweighting did not converge.
